In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

source_path = '/content/drive/MyDrive/a_colab/MOT20.zip' # Corrected path
destination_path = '/content/MOT20.zip'

# Ensure the source file exists before attempting to copy
if os.path.exists(source_path):
    !cp "{source_path}" "{destination_path}"
    print(f"File copied from {source_path} to {destination_path}")
else:
    print(f"Error: Source file not found at {source_path}")

File copied from /content/drive/MyDrive/a_colab/MOT20.zip to /content/MOT20.zip


In [ ]:
!unzip /content/MOT20.zip

Archive:  /content/MOT20.zip
replace __MACOSX/._MOT20? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._MOT20        
replace __MACOSX/MOT20/._test? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls /content/MOT20
!ls /content/MOT20/train/
!ls /content/MOT20/test/

test  train
MOT20-01  MOT20-02  MOT20-03  MOT20-05
MOT20-04  MOT20-06  MOT20-07  MOT20-08


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.1 MB/s eta 0:00:00


In [ ]:
# !pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 52.4/112.6 GB disk)


In [ ]:
#convert mot ground truths to yolo
import os
from pathlib import Path

DATASET_RAW = Path("/content/MOT20")
DATASET_YOLO = Path("/content/mot20_yolo")

os.makedirs(DATASET_YOLO, exist_ok=True)

In [ ]:
import os, shutil, cv2
from pathlib import Path

DATASET_RAW = Path("/content/MOT20")
DATASET_YOLO = Path("/content/mot20_yolo")

TRAIN_SEQS = ["MOT20-01", "MOT20-02", "MOT20-03", "MOT20-05"]
# TEST_SEQS  = ["MOT20-04", "MOT20-06", "MOT20-07", "MOT20-08"]

def convert_mot_to_yolo(seq_name):
    seq_src = DATASET_RAW / "train" / seq_name if (DATASET_RAW/"train"/seq_name).exists() else DATASET_RAW / "test" / seq_name
    img_dir = seq_src / "img1"
    gt_file = seq_src / "gt" / "gt.txt"

    seq_dst = DATASET_YOLO / seq_name
    (seq_dst/"images").mkdir(parents=True, exist_ok=True)
    (seq_dst/"labels").mkdir(parents=True, exist_ok=True)

    img_list = sorted(list(img_dir.glob("*.jpg")))
    if not img_list:
        img_list = sorted(list(img_dir.glob("*.png")))
    if not img_list:
        raise ValueError("no images found in " + str(img_dir))

    # determine size
    sample = cv2.imread(str(img_list[0]))
    H, W = sample.shape[:2]

    # copy images
    for img in img_list:
        shutil.copy(img, seq_dst/"images"/img.name)

    # read GT
    with open(gt_file) as f:
        lines = [l.strip() for l in f.readlines()]

    labels = {}
    for l in lines:
        fno, tid, x, y, w, h, conf = l.split(",")[:7]
        fno = int(fno)
        x, y, w, h = map(float, (x, y, w, h))
        conf = float(conf)

        if conf < 0.5:
            continue

        xc = (x + w/2) / W
        yc = (y + h/2) / H
        nw = w / W
        nh = h / H

        line = f"0 {xc:.6f} {yc:.6f} {nw:.6f} {nh:.6f}"
        labels.setdefault(fno, []).append(line)

    # write label files
    for frame, items in labels.items():
        with open(seq_dst/"labels"/f"{frame:06d}.txt", "w") as f:
            f.write("\n".join(items))

    print("done →", seq_name)

for seq in TRAIN_SEQS:
    convert_mot_to_yolo(seq)

print("conversion finished!")


done → MOT20-01
done → MOT20-02
done → MOT20-03
done → MOT20-05
conversion finished!


In [ ]:
!ls /content/mot20_yolo


In [ ]:
import yaml

data_yaml = {
    "path": "/content/mot20_yolo",
    "train": [
        "MOT20-01/images",
        "MOT20-02/images",
        "MOT20-03/images",
        "MOT20-05/images"
    ],
    "val": "MOT20-03/images",   # good validation sequence
    "nc": 1,
    "names": ["person"]
}

with open("/content/mot20_yolo/data.yaml", "w") as f:
    yaml.safe_dump(data_yaml, f)

print("data.yaml created:")
!cat /content/mot20_yolo/data.yaml


In [ ]:
!ls /content
!ls /content/mot20_yolo


In [ ]:
#dont run
from ultralytics import YOLO

model = YOLO("yolov8s.pt")   # tiny enough for T4

model.train(
    data="/content/mot20_yolo/data.yaml",
    epochs=50,
    imgsz=640,
    batch=4,          # low memory
    workers=2,
    cache=False,
    amp=False,
    name="mot20_yolov8s"
)


In [ ]:
!cp /content/best.pt /content/runs/detect/mot20_resume/weights/


cp: cannot create regular file '/content/runs/detect/mot20_resume/weights/': No such file or directory


In [ ]:
from ultralytics import YOLO

# load your checkpoint
model = YOLO("/content/last-10.pt")

# resume training
model.train(
    data="/content/mot20_yolo/data.yaml",
    epochs=50,       # total target epochs (YOLO will know where to continue)
    resume=True
)

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/mot20_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/last-10.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mot20_yolov8s, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d2a105cbe30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
import os

SAVE_DIR = "/content/drive/MyDrive/a_colab/mot"
os.makedirs(SAVE_DIR, exist_ok=True)

print("save directory:", SAVE_DIR)


save directory: /content/drive/MyDrive/a_colab/mot


In [ ]:
import shutil

src = "/content/runs/detect/mot20_yolov8s/weights/best.pt"
dst = f"{SAVE_DIR}/yolov8s_mot20_best-2.pt"

shutil.copy(src, dst)
print("copied best.pt to:", dst)


copied best.pt to: /content/drive/MyDrive/a_colab/mot/yolov8s_mot20_best.pt


In [ ]:
BEST_PT = "/content/runs/detect/mot20_yolov8s/weights/best.pt"  # adjust if needed

from ultralytics import YOLO

model = YOLO(BEST_PT)
model.export(format="onnx", imgsz=640, opset=12)


Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/runs/detect/mot20_yolov8s/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 206ms
Prepared 6 packages in 6.48s
Installed 6 packages in 294ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.19.1
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.77

requirements: AutoUpdate success ✅ 8.0s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX

'/content/runs/detect/mot20_yolov8s/weights/best.onnx'